##This code will visualize the result of finetuned toxic pharapraser model, you can put a toxic sentence then it will be give the translation from the model

In [ ]:
import pandas as pd
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
from tqdm.auto import tqdm, trange
import gc

def cleanup():
    """
    A helpful function to clean all cached batches.
    """
    gc.collect()
    torch.cuda.empty_cache()
#Reading the inputs

toxic_inputs = df_train_toxic[:10]
#Loading the model. For the baseline we used t5_small_model

base_model_name = 't5-small'
model_name = 't5_base_train_10000'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device);
# Paraphrasing preparation with small example

def paraphrase(text, model, n=None, max_length='auto', temperature=0.0, beams=3):
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=temperature,
        repetition_penalty=3.0,
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    if not n and isinstance(text, str):
        return texts[0]
    return texts
print(paraphrase(['you are idiot'], model, temperature=50.0, beams=10))